Notebook

# Climate Displacement

Year
State
Households Inflow (Number of Returns)
Households Outflow (Number of Returns)
Individuals Inflow (Number of Exemptions)
Individuals Outflow (Number of Exemptions)
Chemical
Dam/Levee Break
Drought
Earthquake
Fire
Flood
Human Cause
Hurricane
Ice
Mud/Landslide
Other
snow
Storm
Terrorism
Tornado
Tsunami
Typhoon
Volcano
Water
Winter

In [ ]:
Folders Setup

code 
    notebook.ipynb
    data
        Disasters
            FEMA_dataset.csv
        StateMigration
            
        


In [ ]:
Pseudo Code:

1. Create new FEMA_dataset
    - contains Year, State, Disaster Type
2. 